<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/Detecting_dementia_from_speech_and_transcripts_using_transformers_May242.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Upload your ADReSSo21-diagnosis-train.tgz file to Colab
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [6]:
# Dementia Detection from Speech and Transcripts using Transformers
# Complete Implementation with Multi-Visualizations for Google Colab

# ============================================================================
# STEP 1: INSTALL REQUIRED PACKAGES AND SETUP
# ============================================================================

!pip install transformers torch torchvision torchaudio librosa pandas scikit-learn matplotlib seaborn numpy plotly umap-learn

import os
import tarfile
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import librosa
import librosa.display
from transformers import BertTokenizer, BertModel, ViTModel, ViTFeatureExtractor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import warnings
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
warnings.filterwarnings('ignore')

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Dataset paths
DATASET_PATHS = {
    'diagnosis_train': '/content/drive/MyDrive/Voice/ADReSSo21-diagnosis-train.tgz',
    'progression_train': '/content/drive/MyDrive/Voice/ADReSSo21-progression-train.tgz',
    'progression_test': '/content/drive/MyDrive/Voice/ADReSSo21-progression-test.tgz'
}

print("Available datasets:")
for name, path in DATASET_PATHS.items():
    exists = "✓" if os.path.exists(path) else "✗"
    print(f"{exists} {name}: {path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using device: cuda
Available datasets:
✓ diagnosis_train: /content/drive/MyDrive/Voice/ADReSSo21-diagnosis-train.tgz
✓ progression_train: /content/drive/MyDrive/Voice/ADReSSo21-progression-train.tgz
✓ progression_test: /content/drive/MyDrive/Voice/ADReSSo21-progression-test.tgz
